In [1]:
import json
from collections import defaultdict
from pathlib import Path

import ifcopenshell
import ifcopenshell.util.classification
import pandas as pd

from utils import get_name, get_reference

In [2]:
element_types = json.loads(Path("TotemDatabase/ElementTypes.json").read_text())
element_types = [e["elementType"] for e in element_types]
element_types[0]

{'id': 'ET304746',
 'names': {'en': 'FoG_Cast in situ_Reinforced concrete_LIB_New_01',
  'fr': 'PS_Coulé sur site_Béton armé_BIB_Neuf_01',
  'nl': 'VG_Ter plaatse gestort_Gewapend beton_BIB_Nieuw_01',
  'de': 'not available'},
 'description': {'en': 'Cast in situ_Reinforced concrete (150 mm) | Foam_PUR (130 mm) | Rigid tiles_Glazed ceramic',
  'fr': 'Coulé sur site_Béton armé (150 mm) | Mousse_PUR (130 mm) | Carreaux rigides_Céramique émaillée',
  'nl': 'Ter plaatse gestort_Gewapend beton (150 mm) | Schuim_PUR (130 mm) | Harde tegels_Geglazuurd keramiek',
  'de': 'Cast in situ_Reinforced concrete (150 mm) | Foam_PUR (130 mm) | Rigid tiles_Glazed ceramic'},
 'reference': '(13.)+',
 'elementCategoryId': 'EC7',
 'backendId': 1,
 'uvalue': 0.202001,
 'biogenicCarbonContent': 0.0,
 'thickness': '0.472200000000',
 'environmentalCost': 30.8793,
 'environmentalCostMaterial': 26.4766608,
 'environmentalCostEnergy': 4.4027,
 'signature': 'ZOiDY3f6vIei/4MPHu7whA3yu3y3BDP+J8T/eZb4Tjk=',
 'composit

In [7]:
functional_units = [t["dimensionalParameterId"] for t in element_types]
print(len(element_types))
print(len(functional_units))
for fu in set(functional_units):
    count = len([x for x in functional_units if x == fu])
    print(fu, count)

print(functional_units)
# [t for t in element_types if t["id"] == "ET305092"]

414
414
84 15
79 344
80 55
['79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', '79', 

In [4]:
element_types_dict = defaultdict(list)
for e in element_types:
    r = get_reference(e["reference"])
    element_types_dict[r].append(e)

In [36]:
ifc_paths = [
    "data/Blok O/3D/IFC/BLKO-KAAN-ZZ-ZZ-M-A-0001-Architecture_new.ifc",
    "data/StrijpS/3D/IFC/STRIJP-KAAN-ZZ-ZZ-M-A-0101.ifc",
    "data/Lumiere/3D/IFC/Lumiere_KAAN_Bouwkundig.ifc",
    "data/Overhoeks/3D/IFC/OHG_B_BWK_KAA_Bouwkundig.ifc",
    "data/SPOT/3D/IFC/SPOT_X_KAAN_BK_B_A1_R20.ifc",
    "data/SPOT/3D/IFC/SPOT_X_KAAN_BK_B_A2_R20.ifc",
    "data/SPOT/3D/IFC/SPOT_X_KAAN_BK_B_B2_R20.ifc",
    "data/SPOT/3D/IFC/SPOT_X_KAAN_BK_B_K_R20.ifc",
    "data/Zalmhaven/3D/IFC/ZHA-B-BWK-B-MR-R18.ifc",
]

In [6]:
product_references = {}
for ifc_path in ifc_paths:
    model = ifcopenshell.open(ifc_path)
    for product in model.by_type("IfcProduct"):
        if product.Representation is None or product.is_a("IfcSpace"):
            continue
        if product.Name is not None:
            name = get_name(product)
            references = ifcopenshell.util.classification.get_references(product)
            if len(references) > 0:
                reference = list(references)[0][1]
                reference = reference.split(".")
                if reference[0] in element_types_dict:
                    product_references[name] = reference

In [7]:
len(product_references)

797

In [8]:
product_references

{'Basic Wall:K_20_WA_GF pigmented wall placeholder': ['22', '00'],
 'Basic Wall:K_22_WA_Concrete in-situ': ['22', '21'],
 'Floor:K_23_FL_Grass placeholder': ['23', '00'],
 'Floor:K_23_FL_Parking': ['23', '00'],
 'Basic Wall:K_21_WA_Housing facade placeholder': ['21', '00'],
 'Basic Wall:K_20_WA_Generic': ['22', '00'],
 'Basic Wall:K_21_WA_Concrete precast': ['21', '21'],
 'Floor:K_23_FL_Concrete in-situ': ['23', '21'],
 'Basic Wall:K_21_WA_Housing facade insulation and alu finish': ['21', '00'],
 'Floor:K_23_FL_Balcony_Lower levels': ['23', '00'],
 'Basic Wall:K_21_WA_Housing facade insulation and stone finish': ['21', '00'],
 'Floor:K_23_FL_Balcony_Generic': ['23', '00'],
 'Basic Wall:K_20_WA_Balustrade_pigmented': ['22', '00'],
 'K_28_SCO_TLB_Generic': ['28', '11'],
 'K_30_WIN_WB_Generic_2 panes_vertical:WI.xxxx_2000x2450mm_housing': ['31',
  '21'],
 'K_32_DO_WB': ['32', '31'],
 'Basic Wall:K_20_WA_Ceramic_300mm': ['22', '00'],
 'K_30_WIN_WB_Generic_1 pane_placeholder:WI.xxxx_1000x24

In [14]:
columns = [
    "Product Ref",
    "Product Name",
    "Selected",
    "Element ID",
    "Element Ref",
    "Element Name",
    "Element Environmental Score",
    "Element Description",
]
data = []
for n, r in product_references.items():
    if r[0] in element_types_dict:
        row = [".".join(r), n] + ([""] * 5)
        data.append(row)
        for e in element_types_dict[r[0]]:
            row = ([""] * 3) + [
                e["id"],
                e["reference"],
                e["names"]["en"],
                e["environmentalCost"],
                e["description"]["en"],
            ]
            data.append(row)

In [15]:
df = pd.DataFrame(data, columns=columns)

In [16]:
df

,Product Ref,Product Name,Selected,Element ID,Element Ref,Element Name,Element Environmental Score,Element Description
0,22.00,Basic Wall:K_20_WA_GF pigmented wall placeholder,,,,,,None
1,,,,ET304850,(22.8)+,PW_Hollow bricks_Fired clay_LIB_New_01,9.45191,Thick coating_Gypsum plaster | Hollow bricks_F...
2,,,,ET304851,(22.8)+,PW_Hollow blocks_Concrete_LIB_New_01,8.79372,Thick coating_Gypsum plaster | Hollow blocks_C...
3,,,,ET304852,(22.8)+,PW_Hollow bricks_Sand-lime_LIB_New_01,17.7362,Rigid tiles_Glazed ceramic | Hollow bricks_San...
4,,,,ET304853,(22.8)+,PW_Blocks_Gypsum_LIB_New_01,4.88114,Blocks_Gypsum (501x100x666 mm) | Blanket_Stone...
...,...,...,...,...,...,...,...,...
30434,,,,ET305145,(22.1)+,Lblw_Solid bricks_Fired clay_LIB_Reno_01,0.95254,Thick coating_Gypsum plaster | Solid bricks_Fi...
30435,,,,ET305146,(22.1)+,Lblw_Solid bricks_Fired clay_LIB_Reno_02,0.642323,Thick coating_Clay plaster | Solid bricks_Fire...
30436,,,,ET305147,(22.3)+,nLblW_Blocks_Hempcrete_LIB_new_01,3.50401,Thick coating_Clay plaster | Blocks_Hempcrete ...
30437,,,,ET305148,(22.3)+,nLblW_Solid Bricks_Fired clay_LIB_Reno_01,1.67243,Thick coating_Gypsum plaster | Solid bricks_Fi...


In [17]:
df.to_csv(Path("totem_mapping.csv"), index=False)